Markdown language

In [57]:
# Dependencies and Setup
import pandas as pd
import math

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

#Create a dataframe to view the initial data
purchase_data_df = pd.DataFrame(purchase_data)

## Player Count

In [58]:
#find 
unique_id = purchase_data["SN"].unique()
unique_id_count = len(unique_id)
unique_id_count_df = pd.DataFrame({"Total Players": [unique_id_count]})
unique_id_count_df

,Total Players
0,576


## Purchasing Analysis (Total)

In [59]:
#number of unique items, average price, number of purchases, total revenue
unique_items = len(purchase_data["Item Name"].unique())
average_price = purchase_data["Price"].mean()
number_of_purchases = len(purchase_data)
total_revenue = purchase_data["Price"].sum()

summary_items = pd.DataFrame(
    {"Number of Unique Items": [unique_items],
     "Average Price": [average_price],
     "Number of Purchases": [number_of_purchases],
     "Total Revenue": [total_revenue]
     }
)
summary_items["Average Price"] = summary_items["Average Price"].map("${:.2f}".format)
summary_items["Total Revenue"] = summary_items["Total Revenue"].map("${:.2f}".format)

summary_items

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


## Gender Demographics

In [60]:
#first I dropped the duplicates of the SNs before counting the gender value counts of each SN
purchase_dropdup = purchase_data_df.drop_duplicates(subset="SN")
gender_count = purchase_dropdup["Gender"].value_counts()
gender_pct = gender_count / unique_id_count * 100
gender_pct_df = pd.DataFrame({"Gender Totals": gender_count, "Gender Percentage": gender_pct})

# Second, I formatted the percentages to round to 2 decimals and include the % symbol
gender_pct_df["Gender Percentage"] = gender_pct_df["Gender Percentage"].map("{:.2f}%".format)

gender_pct_df


,Gender Totals,Gender Percentage
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

In [61]:
gender_gb = purchase_data_df.groupby(["Gender"])
gender_pc = gender_gb["Purchase ID"].count()
gender_app = gender_gb["Price"].mean()
gender_tpv = gender_gb["Price"].sum()
gender_atppp = gender_tpv / gender_pct_df["Gender Totals"]

gender_analysis = pd.DataFrame({"Purchase Count": gender_pc, 
                               "Average Purchase Price": gender_app.map("${:.2f}".format), 
                               "Total Purchase Value": gender_tpv.map("${:.2f}".format), 
                               "Avg Total Purchase per Person": gender_atppp.map("${:.2f}".format)})
gender_analysis.head(20)



,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

In [62]:
## dynamic bins - this will create a dynamic bins method

# step 1: create bins list
# start bins at 0, 10 - the minimum needed (if ages are less than 10)
bins = [0, 9]
list_appender = 9

In [63]:
## find the maximum age, then determine iterations by rounding up to the nearest 5 
# NOTE: (nothing changes if already divisible by 5 i.e. 20, 35, etc)
large_age = purchase_data_df["Age"].max()
iterations = math.ceil((large_age + 1) / 5) - 2

In [64]:
# skip the for loop to add anything if max number is less than 10...
# otherwise add a number 5 greater than previous list_appender to create bins (for x-interations)
if large_age < 10 :
    bins = [0, 9]
else:
    for x in range(iterations):
        bins.append(list_appender + 5)
        list_appender += 5

In [65]:
# step 2: create age_group list
# sets first label to "<10" (for when bins is just [0, 10]), then add string to list for x-iterations
age_groups = ["<10"]
for x in range(iterations):
    age_groups.append(f"{bins[x+1]+1}-{bins[x+1]+5}")

In [66]:
#create bins for Age Groups by cutting purchase data Df 
age_stats = pd.cut(purchase_data_df["Age"], bins, labels=age_groups, include_lowest=True)
purchase_data_df["Age Groups"] = pd.cut(purchase_data_df["Age"], bins, labels=age_groups)
age_group_df = purchase_data_df
age_grouping = purchase_data_df.groupby("Age Groups")


#first I dropped the duplicates of the SNs before counting the gender value counts of each SN
purchase_dropdup = purchase_data_df.drop_duplicates(subset="SN")
age_count = purchase_dropdup["Age Groups"].value_counts()
age_pct = age_count / unique_id_count * 100
age_pct_df = pd.DataFrame({"Age Totals": age_count, "Age Percentage": age_pct})

# # Second, I formatted the percentages to round to 2 decimals and include the % symbol
age_pct_df["Age Percentage"] = age_pct_df["Age Percentage"].map("{:.2f}%".format)
age_pct_sorted = age_pct_df.sort_index()
age_pct_sorted.head()

,Age Totals,Age Percentage
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%


## Purchasing Analysis (Age)

In [67]:
age_gb = purchase_data_df.groupby(["Age Groups"])
age_pc = age_gb["Purchase ID"].count()
age_app = age_gb["Price"].mean()
age_tpv = age_gb["Price"].sum()
age_atppp = age_tpv / age_pct_sorted["Age Totals"]

age_analysis_df = pd.DataFrame({"Purchase Count": age_pc.map("{:.0f}".format), 
                               "Average Purchase Price": age_app.map("${:.2f}".format), 
                               "Total Purchase Value": age_tpv.map("${:.2f}".format), 
                               "Avg Total Purchase per Person": age_atppp.map("${:.2f}".format),
                               })

age_analysis_df.head(20)

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Groups,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40-44,12,$3.04,$36.54,$3.32
45-49,1,$1.70,$1.70,$1.70


## Top Spenders

In [68]:
player_gb = purchase_data_df.groupby(["SN"])
player_pc = player_gb.count()["Purchase ID"]
player_app = player_gb.mean()["Price"]
player_tpv = player_gb.sum()["Price"]
player_analysis_df = pd.DataFrame({"Purchase Count": player_pc.map("{:.0f}".format), 
                               "Average Purchase Price": player_app.map("${:.2f}".format), 
                               "Total Purchase Value": player_tpv, 
                               })
player_analysis_df = player_analysis_df.sort_values("Total Purchase Value", ascending=False)
player_analysis_df["Total Purchase Value"] = player_analysis_df["Total Purchase Value"].map("${:.2f}".format)

player_analysis_df.head(5)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

In [69]:
item_retrieval_df = pd.DataFrame({"Item ID": purchase_data_df["Item ID"], 
                               "Item Name": purchase_data_df["Item Name"], 
                               "Item Price": purchase_data_df["Price"], 
                               })

item_gb = item_retrieval_df.groupby(["Item ID", "Item Name"])
item_pc = item_gb.count()["Item Price"]
item_app = item_gb.mean()["Item Price"]
item_tpv = item_gb.sum()["Item Price"]
item_analysis_df = pd.DataFrame({"Purchase Count": item_pc, 
                               "Item Price": item_app, 
                               "Total Purchase Value": item_tpv, 
                               })
item_popular_df = item_analysis_df.sort_values("Purchase Count", ascending=False)
item_popular_df["Item Price"] = item_analysis_df["Item Price"].map("${:.2f}".format)
item_popular_df["Total Purchase Value"] = item_analysis_df["Total Purchase Value"].map("${:.2f}".format)

item_popular_df.head(5)


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

In [70]:
item_profitable_df = item_analysis_df.sort_values("Total Purchase Value", ascending=False)
item_profitable_df["Item Price"] = item_analysis_df["Item Price"].map("${:.2f}".format)
item_profitable_df["Total Purchase Value"] = item_analysis_df["Total Purchase Value"].map("${:.2f}".format)

item_profitable_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
